In [1]:
import os
import numpy as np
import cv2
from pyswarm import pso

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from pyswarm import pso
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Paths ---
csv_path = "D:/B.Tech/8th Sem/CI Lab/Project/Cylone_Intensity_IndianOcean/label.csv"
image_dir = "D:/B.Tech/8th Sem/CI Lab/Project/Cylone_Intensity_IndianOcean/Infrared"

In [3]:
# --- 1. Load CSV ---
df = pd.read_csv(csv_path)
df['filepath'] = df['Filename'].apply(lambda x: os.path.join(image_dir, x))
print(f"Total samples from CSV: {len(df)}")

Total samples from CSV: 15101


In [4]:
# --- 2. Extract raw features from images ---
def extract_image_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 64)).astype(np.float32) / 255.0
    return image.flatten()

X = np.array([extract_image_features(p) for p in df['filepath']])
y = df['Vmax'].values

print(f"Raw feature shape: {X.shape}")  # (samples, features)
print(f"Labels shape: {y.shape}")

Raw feature shape: (15101, 4096)
Labels shape: (15101,)


In [5]:
# --- 4. Load saved selected feature mask ---
selected_mask = np.load("selected_features_mask.npy")
X_selected = X[:, selected_mask]
print(f"Selected features count: {np.sum(selected_mask)}")
print(f"Feature shape after PSO selection: {X_selected.shape}")

# --- 5. Train-test split ---
# split: train and test
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42)


# Print shapes to confirm
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

Selected features count: 2141
Feature shape after PSO selection: (15101, 2141)
X_test: (3021, 2141), y_test: (3021,)
X_train: (12080, 2141), y_train: (12080,)


In [6]:
# --- Convert to Tensors ---
def to_loader(X, y):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    return DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=32, shuffle=True)

train_loader = to_loader(X_train, y_train)
test_loader = to_loader(X_test, y_test)


## Linear Regression without Regularization

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Train Linear Regression model ---
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

LinearRegression()

In [9]:
# --- Predict ---
y_pred_train = lr_model.predict(X_train_scaled)

# --- Train Evaluation ---
mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print(f"Train Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Train Evaluation Results:
Mean Absolute Error (MAE): 18.46
Mean Squared Error (MSE): 573.11
R2 Score: 0.5183


In [10]:
# --- Test Evaluation ---
y_pred = lr_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Test Evaluation Results:
Mean Absolute Error (MAE): 22.62
Mean Squared Error (MSE): 839.75
R2 Score: 0.2840


## Ridge Regression (L2 regularization)

In [7]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge_model = Ridge(alpha=1.0)  # alpha is the regularization strength
ridge_model.fit(X_train_scaled, y_train)

Ridge()

In [8]:
# --- Predict ---
y_pred_train = ridge_model.predict(X_train_scaled)

# --- Train Evaluation ---
mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print(f"Train Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Train Evaluation Results:
Mean Absolute Error (MAE): 18.46
Mean Squared Error (MSE): 573.11
R2 Score: 0.5183


In [9]:
# --- Test Evaluation ---
y_pred = ridge_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Test Evaluation Results:
Mean Absolute Error (MAE): 22.61
Mean Squared Error (MSE): 839.12
R2 Score: 0.2845


## Lasso Regression (L1 regularization)

In [8]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso_model = Lasso(alpha=0.1, max_iter=10000)
lasso_model.fit(X_train_scaled, y_train)

Lasso(alpha=0.1, max_iter=10000)

In [9]:
# --- Predict ---
y_pred_train = lasso_model.predict(X_train_scaled)

# --- Train Evaluation ---
mae = mean_absolute_error(y_train, y_pred_train)
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print(f"Train Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Train Evaluation Results:
Mean Absolute Error (MAE): 19.82
Mean Squared Error (MSE): 657.05
R2 Score: 0.4478


In [10]:
# --- Test Evaluation ---
y_pred = lasso_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

Test Evaluation Results:
Mean Absolute Error (MAE): 20.72
Mean Squared Error (MSE): 722.33
R2 Score: 0.3841


## Grid Search for Ridge

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# --- Grid search for alpha ---
param_grid = {'alpha': [0.01, 0.1, 1.0, 10, 100, 1000]}
ridge_base  = Ridge()
grid = GridSearchCV(ridge_base, param_grid,
                    cv=5, scoring='neg_mean_squared_error',
                    n_jobs=-1, verbose=2)
grid.fit(X_train_scaled, y_train)

print("Best alpha:", grid.best_params_['alpha'])
best_model = grid.best_estimator_        # <<< tuned & already fitted

# ------------- Evaluation helper -------------
def evaluate(split, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    print(f"{split:<5} | MAE: {mae:.3f}  MSE: {mse:.3f}  R²: {r2:.4f}")

# --- Predict & report ---
y_pred_train = best_model.predict(X_train_scaled)
y_pred_test  = best_model.predict(X_test_scaled)

evaluate("Train", y_train, y_pred_train)
evaluate("Test ", y_test,  y_pred_test)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best alpha: 1000
Train | MAE: 19.184  MSE: 618.642  R²: 0.4801
Test  | MAE: 21.132  MSE: 747.369  R²: 0.3628
